# GTIN to ePID

## Summary
The goal of this project is to obtain the eBay Product ID(ePID) from a products Global Trade Item Number(GTIN). A list of random GTINs will be used.

<b>GTIN</b>: "GTIN describes a family of GS1 (EAN.UCC) global data structures that employ 14 digits and can be encoded into various types of data carriers. Currently, GTIN is used exclusively within bar codes, but it could also be used in other data carriers such as radio frequency identification (RFID). The GTIN is only a term and does not impact any existing standards, nor does it place any additional requirements on scanning hardware. For North American companies, the UPC is an existing form of the GTIN." <sup>1</sup>  
<b>ePID</b>: "eBay Product ID. eBay's global reference ID for a catalog product. On the eBay Web site, this is known as the "ePID". A reference ID is a fixed reference to a product (regardless of version). One reference ID can be associated with multiple ProductID values. The value should start with "EPID", for example "EPID30971438" (without quotes). You can find reference IDs for products by using GetProducts (or FindProducts in the Shopping API). You can also find the ePID on eBay's Web site." <sup>2</sup>

## Import Libraries and Modules

In [260]:
# import libraries and modules
import pandas as pd # data manipulation and analysis library
import numpy as np # scientific computing package
import requests # HTTP library
import re # Regular expression operations module
from bs4 import BeautifulSoup # HTML and XML document parsing package
import time # Time access and conversions module

## Get list of GTINs as Pandas DataFrame
List of ASIN/GTINs is hosted on Google Drive as a CSV file. URL has been shortened for convenience. The path could be changed as needed.

In [261]:
# download list
!wget -O gtin_needs_epid.csv 'https://bit.ly/2AZL0LY'
print('Download complete!')

--2019-01-23 12:55:33--  https://bit.ly/2AZL0LY
Resolving bit.ly... 67.199.248.11, 67.199.248.10
Connecting to bit.ly|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://docs.google.com/spreadsheets/d/e/2PACX-1vSnchQ-ctWClVLy1R8xJhAhCHg7OgMqJCqukvhJNJJBQgbOGCcLDXKSdXeNSxG5XPC6n3DWphYp-bsZ/pub?gid=1781862218&single=true&output=csv [following]
--2019-01-23 12:55:34--  https://docs.google.com/spreadsheets/d/e/2PACX-1vSnchQ-ctWClVLy1R8xJhAhCHg7OgMqJCqukvhJNJJBQgbOGCcLDXKSdXeNSxG5XPC6n3DWphYp-bsZ/pub?gid=1781862218&single=true&output=csv
Resolving docs.google.com... 172.217.2.14
Connecting to docs.google.com|172.217.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘gtin_needs_epid.csv’

gtin_needs_epid.csv     [ <=>                ]   4.56K  --.-KB/s    in 0.002s  

2019-01-23 12:55:34 (2.94 MB/s) - ‘gtin_needs_epid.csv’ saved [4673]

Download complete!


In [262]:
# create pandas DataFrame with list from CSV file
gtin_df = pd.read_csv('gtin_needs_epid.csv', header=0, sep=',', dtype='str')
print(gtin_df.head()) # preview DataFrame
print(gtin_df.shape) # print DataFrame Shape

         ASIN           UPC            EAN ISBN Missing
0  B004XC6GJ0  021111531467  0168141495601  NaN   False
1  B000P1DEHU  027242708952  0027242708952  NaN   False
2  B004NEUJKA  961613210272  0999993504940  NaN   False
3  B0011ULQNI           NaN            NaN  NaN    True
4  B001O0DP48  050633271667  0050633271667  NaN   False
(107, 5)


In [263]:
# Replace NaN values with blank values
gtin_df = gtin_df.replace(np.nan, '', regex=True)
gtin_df.head()

,ASIN,UPC,EAN,ISBN,Missing
0,B004XC6GJ0,021111531467,0168141495601,,False
1,B000P1DEHU,027242708952,0027242708952,,False
2,B004NEUJKA,961613210272,0999993504940,,False
3,B0011ULQNI,,,,True
4,B001O0DP48,050633271667,0050633271667,,False


## Scrape Data from eBay Search Results
This scraping technique will query the standard web-based search of eBay with each GTIN value. If the GTIN value matches a product stored in eBay's Product Catalog, then the results typically provide a link to a product reviews page. The product review page URL includes the associated ePID. The ePID will be parsed from the URL and stored within a DataFrame. There will be a DataFrame for each type of GTIN before being aggregated.

In [264]:
# function for obtaining ePID
def gtin_to_ebay_url(gtin):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        r = requests.get('https://www.ebay.com/sch/i.html?&_nkw={}'.format(gtin), headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        review_url = soup.find_all("div", class_='s-item__reviews')[0]
        for a in review_url.find_all('a', href=True):
            review_url = a['href']
        review_url = re.sub('/p/(.*)/', '/p/', review_url, flags=0)
        review_url = re.sub('\?iid=(.*)', '', review_url, flags=0)
        review_url = re.sub('https://(.*)/p/', '', review_url, flags=0)
        return(review_url)
    except:
        return('')

In [265]:
# loop function
def gtin_loop(df, gtin_type):
    asin_epid_df = pd.DataFrame(columns=['ASIN', '{}_ePID'.format(gtin_type)]) # Constructing a DataFrame for ePID data by ASIN
    for i, gtin in enumerate(df[gtin_type]):
        print('Processing {} of {}: {}'.format(str(i + 1).zfill(3), len(df[gtin_type]), gtin))
        if gtin != '':
            time.sleep(2) # limits number of requests per minute to prevent being blacklisted
            epid = gtin_to_ebay_url(gtin)
        else:
            epid = ''
        epid_dict = {'ASIN': df['ASIN'].loc[i], '{}_ePID'.format(gtin_type): epid}
        asin_epid_df = asin_epid_df.append(epid_dict, ignore_index=True)
    return(asin_epid_df)

In [266]:
# Use UPC as identifier
upc_epid_df = gtin_loop(gtin_df, 'UPC')

Processing 001 of 107: 021111531467
Processing 002 of 107: 027242708952
Processing 003 of 107: 961613210272
Processing 004 of 107: 
Processing 005 of 107: 050633271667
Processing 006 of 107: 037988482528
Processing 007 of 107: 
Processing 008 of 107: 050633151624
Processing 009 of 107: 079000309598
Processing 010 of 107: 050633320129
Processing 011 of 107: 031752160506
Processing 012 of 107: 829646543111
Processing 013 of 107: 
Processing 014 of 107: 741725317195
Processing 015 of 107: 634173081957
Processing 016 of 107: 
Processing 017 of 107: 
Processing 018 of 107: 
Processing 019 of 107: 723905218522
Processing 020 of 107: 
Processing 021 of 107: 666230470992
Processing 022 of 107: 609585151833
Processing 023 of 107: 197274678803
Processing 024 of 107: 689227634450
Processing 025 of 107: 606449066456
Processing 026 of 107: 
Processing 027 of 107: 
Processing 028 of 107: 
Processing 029 of 107: 045496860219
Processing 030 of 107: 
Processing 031 of 107: 806293905844
Processing 032 o

In [267]:
upc_epid_df.head()

,ASIN,UPC_ePID
0,B004XC6GJ0,1137898396
1,B000P1DEHU,61650824
2,B004NEUJKA,
3,B0011ULQNI,
4,B001O0DP48,


In [268]:
# Use EAN as identifier
ean_epid_df = gtin_loop(gtin_df, 'EAN')

Processing 001 of 107: 0168141495601
Processing 002 of 107: 0027242708952
Processing 003 of 107: 0999993504940
Processing 004 of 107: 
Processing 005 of 107: 0050633271667
Processing 006 of 107: 0037988482528
Processing 007 of 107: 
Processing 008 of 107: 0050633151624
Processing 009 of 107: 0079000309598
Processing 010 of 107: 0050633320129
Processing 011 of 107: 0031752160506
Processing 012 of 107: 0801964781178
Processing 013 of 107: 
Processing 014 of 107: 0741725317195
Processing 015 of 107: 0634173081957
Processing 016 of 107: 
Processing 017 of 107: 5704327760481
Processing 018 of 107: 5704327232209
Processing 019 of 107: 0723905218522
Processing 020 of 107: 5060301841240
Processing 021 of 107: 0666230470992
Processing 022 of 107: 0609585151833
Processing 023 of 107: 0666672977165
Processing 024 of 107: 0689227634450
Processing 025 of 107: 0606449066456
Processing 026 of 107: 
Processing 027 of 107: 
Processing 028 of 107: 
Processing 029 of 107: 0045496860219
Processing 030 of 

In [269]:
ean_epid_df.head()

,ASIN,EAN_ePID
0,B004XC6GJ0,513019118
1,B000P1DEHU,61650824
2,B004NEUJKA,
3,B0011ULQNI,
4,B001O0DP48,77183800


In [270]:
# Use ISBN as identifier
isbn_epid_df = gtin_loop(gtin_df, 'ISBN')

Processing 001 of 107: 
Processing 002 of 107: 
Processing 003 of 107: 
Processing 004 of 107: 
Processing 005 of 107: 
Processing 006 of 107: 
Processing 007 of 107: 
Processing 008 of 107: 
Processing 009 of 107: 
Processing 010 of 107: 
Processing 011 of 107: 
Processing 012 of 107: 
Processing 013 of 107: 
Processing 014 of 107: 
Processing 015 of 107: 
Processing 016 of 107: 
Processing 017 of 107: 
Processing 018 of 107: 
Processing 019 of 107: 
Processing 020 of 107: 
Processing 021 of 107: 
Processing 022 of 107: 
Processing 023 of 107: 
Processing 024 of 107: 
Processing 025 of 107: 
Processing 026 of 107: 
Processing 027 of 107: 
Processing 028 of 107: 
Processing 029 of 107: 
Processing 030 of 107: 
Processing 031 of 107: 
Processing 032 of 107: 
Processing 033 of 107: 
Processing 034 of 107: 
Processing 035 of 107: 
Processing 036 of 107: 
Processing 037 of 107: 
Processing 038 of 107: 
Processing 039 of 107: 
Processing 040 of 107: 
Processing 041 of 107: 
Processing 042 o

In [271]:
isbn_epid_df.head()

,ASIN,ISBN_ePID
0,B004XC6GJ0,
1,B000P1DEHU,
2,B004NEUJKA,
3,B0011ULQNI,
4,B001O0DP48,


In [284]:
# merge *_epid_dfs
asin_epid_merged_df = upc_epid_df.merge(ean_epid_df, on='ASIN')
asin_epid_merged_df = asin_epid_merged_df.merge(isbn_epid_df, on='ASIN')
asin_epid_merged_df.head()

,ASIN,UPC_ePID,EAN_ePID,ISBN_ePID
0,B004XC6GJ0,1137898396,513019118,
1,B000P1DEHU,61650824,61650824,
2,B004NEUJKA,,,
3,B0011ULQNI,,,
4,B001O0DP48,,77183800,


In [286]:
# Replace NaN values with blank values
asin_epid_merged_df = asin_epid_merged_df.replace(np.nan, '', regex=True)
asin_epid_merged_df.head()

,ASIN,UPC_ePID,EAN_ePID,ISBN_ePID
0,B004XC6GJ0,1137898396,513019118,
1,B000P1DEHU,61650824,61650824,
2,B004NEUJKA,,,
3,B0011ULQNI,,,
4,B001O0DP48,,77183800,


## Indicate Completeness of Data
It is desireable to have at least one ePID value present for UPC, EAN, or ISBN. A boolean value will be assiged to a new column named "Missing". If the value is True, then the ASIN of that row is lacking ePID values for UPC, EAN, and ISBN.

In [287]:
# Construct DataFrame for Missing value
missing_data_columns = ['ASIN','Missing']
asin_epid_missing_df = pd.DataFrame(columns = missing_data_columns)
asin_epid_missing_df

,ASIN,Missing


In [288]:
# loop to record missing value for each ASIN
missing_data = []
for row in asin_epid_df.iterrows():
    missing = True
    if row[1][1] != '':
        missing = False
    elif row[1][2] != '':
        missing = False
    elif row[1][3] != '':
        missing = False
    missing_dict = {'ASIN': row[1][0], 'Missing' : missing}
    asin_epid_missing_df = asin_epid_missing_df.append(missing_dict, ignore_index=True)

In [289]:
asin_epid_missing_df.head()

,ASIN,Missing
0,B004XC6GJ0,False
1,B000P1DEHU,False
2,B004NEUJKA,True
3,B0011ULQNI,True
4,B001O0DP48,False


In [291]:
# merge DataFrames
asin_epid_merged_df = asin_epid_merged_df.merge(asin_epid_missing_df, on='ASIN')
# preview DataFrame
asin_epid_merged_df.head()

,ASIN,UPC_ePID,EAN_ePID,ISBN_ePID,Missing
0,B004XC6GJ0,1137898396,513019118,,False
1,B000P1DEHU,61650824,61650824,,False
2,B004NEUJKA,,,,True
3,B0011ULQNI,,,,True
4,B001O0DP48,,77183800,,False


## Results

In [294]:
# Count how many ASINs are missing a GTIN
missing_count = asin_epid_merged_df[asin_epid_merged_df['Missing'] == True].shape[0]
total_count = asin_epid_merged_df.shape[0]
print('Number of ASINs lacking an ePID: {}'.format(missing_count))
print('Success rate: {}%'.format((total_count - missing_count) / total_count * 100))

Number of ASINs lacking an ePID: 76
Success rate: 28.971962616822427%


## Append Data to a Tab-separated values file
Tab-separated values are preferable to Comma-separated values when pasting data to most spreadsheet applications. The mode is set to 'a' so that the resulting data is added to existing data rather than overwriting the existing data. This process might be used continually where the TSV file continues to grow.

In [295]:
asin_epid_merged_df.to_csv('asin_epid.tsv', sep='\t' , index=False, mode='a')
print('Export complete.')

Export complete.


## References
1. GTIN Info: Global Trade Item Number. (n.d.). Retrieved January 23, 2019, from https://www.gtin.info/
2. ProductIdentifier. (2016). Retrieved January 23, 2019, from https://developer.ebay.com/devzone/product/CallRef/types/ProductIdentifier.html